In [1]:
import torch
import transformers
from lora import *

import warnings
warnings.filterwarnings('always')

# Preparing model and data

In [2]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
text = "Sample text"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Note: ```transformers``` are saying that our model final dense layer is not initialized from checkpoint, which makes sense since for our task this layer is supposed to be completely fine-tuned on down-stream task

In [11]:
print(output.last_hidden_state.shape) # output: BaseModelOutputWithPoolingAndCrossAttentions

torch.Size([1, 12, 768])


In [16]:
print(model.encoder.layer[0].attention) # inside the MHA (we are working with this stuff)

RobertaAttention(
  (self): RobertaSelfAttention(
    (query): Linear(in_features=768, out_features=768, bias=True)
    (key): Linear(in_features=768, out_features=768, bias=True)
    (value): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (output): RobertaSelfOutput(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


We will follow LoRA-FA experiment setting, where in paper they state that they apply lora to all 4 linear layers in MultiHeadAttention (see section 2.2): 3 layers that produce $Q$, $K$ and $V$ and also final linear layer weights

<center>
<img src="imgs/mha.png" width=300 height=400/>
</center>

In [ ]:
# probably generated like in stupid_demo
# but it can be done simpler, since every RobertaSelfAttention and RobertaSelfOutput (see output earlier)
# has all the needed linear layers, so no need to search for them directly in config generation
lora_roberta_config = {
    ...
}

# Fine-tuning stage